# DataRoom API Client Examples

# Initialize the client

In [ ]:
import os
from dataroom_client import DataRoomClient

# You can specify the API key with an environment variable
os.environ['DATAROOM_API_KEY'] = 'YOUR_KEY_HERE'

# or by passing them directly to the DataRoomClient
DataRoom = DataRoomClient(
    api_url='http://localhost:8000/api/',
)

#### Utils

In [ ]:
import numpy as np
import random

def get_random_vector():
    vector = np.array([random.randint(-99, 99) / 100 for _ in range(768)])
    norm = np.linalg.norm(vector)
    if norm == 0: 
        return get_random_vector()
    normalized_vector = vector / norm
    return normalized_vector

# Images

### Create an image from URL
If an existing image with the same URL exists, it will be returned instead of creating a new one.

In [ ]:
urls = [
    'https://storyblok-cdn.photoroom.com/f/191576/1200x800/e5ea306bf8/instant_backgrounds_before_.webp',
    'https://storyblok-cdn.photoroom.com/f/191576/1200x800/88361a19c7/retouch_before_.webp',
    'https://storyblok-cdn.photoroom.com/f/191576/1200x800/56d7182862/blur_background_before_.webp',
    'https://storyblok-cdn.photoroom.com/f/191576/1200x800/766477a3f1/instant_shadows_before_.webp',
    'https://storyblok-cdn.photoroom.com/f/191576/1200x800/f647012ecc/black_background_before_.webp',
    'https://storyblok-cdn.photoroom.com/f/191576/1200x800/395dec599c/add_text_to_photo_before_.webp',
    'https://storyblok-cdn.photoroom.com/f/191576/1200x800/df26425c7c/transparent_background_before_.webp',
    'https://storyblok-cdn.photoroom.com/f/191576/1200x800/90c0650cff/add_background_before_.webp',
    'https://storyblok-cdn.photoroom.com/f/191576/1200x800/79462dda37/profile_picture_maker_before_.webp',
    'https://storyblok-cdn.photoroom.com/f/191576/1200x800/261d04cfee/color_splasher_before_.webp',
    'https://storyblok-cdn.photoroom.com/f/191576/1200x800/e11629f859/motion_blur_effect_before_-1.webp',
    'https://storyblok-cdn.photoroom.com/f/191576/1176x882/c6f7f639fb/change_color_of_image_before_.webp',
    'https://storyblok-cdn.photoroom.com/f/191576/1200x800/796a3001b7/image_brightener_before_-1.webp',
    'https://storyblok-cdn.photoroom.com/f/191576/1200x800/485a1e0863/round_profil_picture_before_.webp',
    'https://storyblok-cdn.photoroom.com/f/191576/1200x800/421076ee50/outline_image_before_-photoroom.jpg',
    'https://storyblok-cdn.photoroom.com/f/191576/1200x800/f886123123/black_and_white_before_.webp',
    'https://storyblok-cdn.photoroom.com/f/191576/1200x800/4e54b928ef/remove_background.webp',
]

for i, url in enumerate(urls):
    image = await DataRoom.create_image(image_id=f'notebook-{i}', image_url=url, source='website')
    print(image['id'])

### Get all the images
The images API is paginated and the client returns the first 1000 images by default. Use the `limit` argument to change the number of images returned.

In [ ]:
images = await DataRoom.get_images()
images

You can specify which fields to return using the `fields` argument.

In [ ]:
images = await DataRoom.get_images(fields=['id'])
images

### Get one specific image

In [ ]:
images = await DataRoom.get_images()
image = None
if len(images):
    image = await DataRoom.get_image(images[0]['id'])
image

### Get images with async generator

In [ ]:
count = 0
page_size = 1000
async for item in DataRoom.get_images_iter(fields=['id'], limit=10000, page_size=page_size):
    count += 1
    if count % page_size == 0:
        print(f'Got {count} images')
print(f'Total: {count} images')

### Get images cached
The first request will set the cache for 60 seconds and all subsequent requests will return the cached data.

In [ ]:
images = await DataRoom.get_images(fields=['id'], cache_ttl=60)
images = await DataRoom.get_images(fields=['id'], cache_ttl=60)

### Create an image from a downloaded file
Here, we download an image from a URL and then pass it to the DataRoomClient as an IOStream. It would be simpler to just pass the URL to the DataRoomClient instead, but this is just an example of how to pass files that are stored locally.

In [ ]:
image_file = await DataRoom.download_image_from_url('https://www.photoroom.com/_next/image?url=%2F_next%2Fstatic%2Fmedia%2Fteam.908694c1.jpg&w=3840&q=75')

image = await DataRoom.create_image(image_id='notebook-from-file-0', image_file=image_file, source='website')
image

### Create an image from a local file
Example of how to create an image from a local file.

In [ ]:
from dataroom_client import DataRoomFile

image_file = DataRoomFile.from_path('image.png')
image = await DataRoom.create_image(image_id='notebook-from-file-1', image_file=image_file, source='local')
image

### Create image with attributes

In [ ]:
from dataroom_client import DataRoomFile

image_file = DataRoomFile.from_path('image.png')
image = await DataRoom.create_image(
    image_id='notebook-from-file-2',
    image_file=image_file,
    source='local',
    attributes={
        'color': 'blue',
        'user': 'john',
    },
)
image

### Create images in bulk

In [ ]:
from dataroom_client import DataRoomFile, DataRoomError

image_paths = [os.path.join('local_files', path) for path in os.listdir('local_files')]
image_files = []
for path in image_paths[:10]:
    try:
        image_files.append(DataRoomFile.from_path(path))
    except DataRoomError as e:
        print(e)

response = await DataRoom.create_images({
    "id": f"bulk-create-{image.path.split('/')[-1].split('.')[0]}",
    "source": "bulk-create",
    "image_file": image,
} for image in image_files)

response

### Update image

In [ ]:
from dataroom_client import DataRoomFile

latent_file = DataRoomFile.from_path('image_latent.txt')
mask_file = DataRoomFile.from_path('image_mask.png')

image = await DataRoom.update_image(
    image_id='0002821a23855cb460ba021a8c1d8ef7',
    source='dataroom',
    attributes={
        'color': 'red',
    },
    tags=['blue', 'red'],
    latents=[{
        "latent_type": "embedding",
        "file": latent_file,
    }, {
        "latent_type": "mask",
        "file": mask_file,
        "is_mask": True,
    }],
)
image

### Tag images

In [ ]:
response = await DataRoom.tag_images(
    image_ids=[
        images[0]['id'],
        images[1]['id'],
        images[2]['id'],
    ],
    tag_names=['tag1', 'tag2'],
)
response

### Add image attributes
Preserves the existing attributes and adds or updates the given ones.

In [ ]:
image = await DataRoom.add_image_attributes(
    image_id='notebook-from-file-2',
    attributes={
        'new_attr': 'new',
    },
)
image

### Add image attributes in bulk

In [ ]:
response = await DataRoom.add_image_attributes_in_bulk({
    image['id']: {
        'new_attr': 'new',
    }
    for image in images[:3]
})
response

### Update images in bulk

In [ ]:
images = await DataRoom.get_images(limit=3, page_size=3)
response = await DataRoom.update_images([
    {
        'id': image['id'],
        'source': 'bulk_update',
    } for image in images
])
response

### Delete image
This permanently deletes the image and all the associated files on S3.

In [ ]:
response = await DataRoom.delete_image(image_id='notebook-from-file-1')
print(response)

### Wrong URL
When providing a wrong URL, the client will raise an exception with the error message.

In [ ]:
image_file = await DataRoom.download_image_from_url('https://www.example.com/does-not-exist.jpg')

In [ ]:
image = await DataRoom.create_image(image_id='fail', image_url='https://www.example.com/does-not-exist.jpg')

### Filter images

In [ ]:
images = await DataRoom.get_images(
    short_edge=882,
)
len(images)

In [ ]:
images = await DataRoom.get_images(
    short_edge__gt=881,
    short_edge__gte=882,
)
len(images)

In [ ]:
images = await DataRoom.get_images(
    short_edge__lt=801,
    short_edge__lte=800,
)
len(images)

In [ ]:
images = await DataRoom.get_images(
    aspect_ratio=1.0,
)
len(images)

In [ ]:
images = await DataRoom.get_images(
    aspect_ratio__lt=2.0,
    aspect_ratio__gte=1.5,
)
len(images)

In [ ]:
images = await DataRoom.get_images(
    aspect_ratio_fraction='4:3',
)
len(images)

In [ ]:
images = await DataRoom.get_images(
    source='website',
)
len(images)

In [ ]:
images = await DataRoom.get_images(
    attributes={
        'color': 'blue',
        'user': 'john',
    },
)
len(images)

In [ ]:
images = await DataRoom.get_images(
    attributes={
        'probability__gt': 100,
    },
)
len(images)

In [ ]:
images = await DataRoom.get_images(
    has_attributes=['color', 'user'],
)
len(images)

In [ ]:
images = await DataRoom.get_images(
    lacks_attributes=['something'],
)
len(images)

In [ ]:
images = await DataRoom.get_images(
    tags=['blue'],
)
len(images)

### Get image logs

In [ ]:
logs = await DataRoom.get_image_audit_logs(images[0]['id'])
logs

### Count images
All the same filters as above can be used to count images.

In [ ]:
count = await DataRoom.count_images(
    aspect_ratio=1.0,
)
print(count)

### Get distance between two images

In [ ]:
images = await DataRoom.get_images()
if len(images) > 1:
    distance = await DataRoom.get_image_distance(images[0]['id'], images[1]['id'])
    print(distance)
else:
    print('Create at least two images first')

### Get similarity between two images

In [ ]:
images = await DataRoom.get_images()
if len(images) > 1:
    similarity = await DataRoom.get_image_similarity(images[0]['id'], images[1]['id'])
    print(similarity)
else:
    print('Create at least two images first')

### Get similar images from id
Get similar images to the image with the given id.
Optionally pass the number of similar images to return.

In [ ]:
images = await DataRoom.get_images()
if len(images):
    similar_images = await DataRoom.get_similar_images(image_id=images[0]['id'], number=2)
    for image in similar_images:
        print(image)
else:
    print('Create at least one image first')

### Get similar images from file
Get similar images to the given image file.

In [ ]:
from dataroom_client import DataRoomFile

image_file = DataRoomFile.from_path('image.png')
similar_images = await DataRoom.get_similar_images(image_file=image_file, number=2)
for image in similar_images:
    print(image)

### Get similar images to vector

In [ ]:
vector = ",".join([str(x) for x in get_random_vector()])
vector = f'[{vector}]'
similar_images = await DataRoom.get_similar_images(
    image_vector=vector,
    number=5,
    fields=['id', 'source'],
    sources=['loader'],
)
for image in similar_images:
    print(image)

### Set image latent
Updates or creates a latent for the image.

In [ ]:
from dataroom_client import DataRoomFile

image = (await DataRoom.get_images(limit=1))[0]

latent_file = DataRoomFile.from_path('image_latent.txt')
response = await DataRoom.set_image_latent(
    image_id=image['id'],
    latent_file=latent_file,
    latent_type='embedding',
)

print(response)

In [ ]:
image = await DataRoom.get_image(image['id'])
image

Delete latent

In [ ]:
response = await DataRoom.delete_image_latent(image_id=image['id'], latent_type='embedding')

Filter by latents

In [ ]:
images = await DataRoom.get_images(
    has_latents=['embedding'],
)
len(images)

In [ ]:
images = await DataRoom.get_images(
    lacks_latents=['embedding'],
)
len(images)

### Get image Coca Embedding
Get an existing Coca Embedding for the image.

In [ ]:
image = await DataRoom.get_image(images[0]['id'], fields=['coca_embedding'])
image['coca_embedding']

### Set image Coca Embedding
Updates or creates a Coca Embedding for the image.

In [ ]:
vector = ",".join([str(x) for x in get_random_vector()])
vector = f'[{vector}]'
embedding = await DataRoom.update_image(images[0]['id'], coca_embedding=vector)
embedding

### Aggregate images

In [ ]:
result = await DataRoom.aggregate_images(
    type='stats',
    field='width',
)
print(result)

### Bucket images

In [ ]:
result = await DataRoom.bucket_images(
    field='width',
    size=10,
)
print(result)

# Synchronous Client
To use the synchronous client, import `DataRoomClientSync` instead of `DataRoomClient`. 
Unfortunately, it is not easily possible to use this client in a notebook.

In [ ]:
from dataroom_client import DataRoomClientSync

DataRoom = DataRoomClientSync(
    api_url='http://localhost:8000/api/',  # no need for prod
)